## P.S. The main idea behind this notebook is inspired from FabienDaniel Kernel Elo_world.
https://www.kaggle.com/fabiendaniel/elo-world

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
warnings.simplefilter(action='ignore', category=FutureWarning)
import gc
import pickle
from sklearn.base import BaseEstimator
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import log_loss
from sklearn.model_selection import RepeatedKFold
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# new_transactions = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv', parse_dates=['purchase_date'])
# historical_transactions = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv', parse_dates=['purchase_date'])

historical_transactions = pd.read_parquet('../input/hist_trans_df.parquet.gzip')
new_transactions = pd.read_parquet('../input/new_trans_df.parquet.gzip')

def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [ ]:
%%time
def add_new_merch_feat(df, merch_df):
    df = pd.merge(left=df,
                  right=merch_df[['merchant_id', 'merchant_group_id', 'category_4']],
                  on=['merchant_id'],
                  how='left')
#     df['year'] = df['purchase_date'].dt.year.astype('int8')
#     df['month'] = df['purchase_date'].dt.month.astype('int8')
    df['dow'] = df['purchase_date'].dt.dayofweek.astype('int8')
    df['hour'] = df['purchase_date'].dt.hour.astype('int8')
    df['day_part'] = 'Morning'
    df.loc[(df['hour'] > 12) & (df['hour'] <= 18), 'day_part'] = 'Afternoon'
    df.loc[(df['hour'] > 18) & (df['hour'] <= 24), 'day_part'] = 'Evening'
    # df['installments_flag'] = np.where(df['installments'] > 1, 'Y', 'N')
    
    # convert into proper data types
#     df['hour'] = df['hour'].astype('category')
#     df['day_part'] = df['day_part'].astype('category')
    # df['installments_flag'] = hist_trans_df['installments_flag'].astype('category')
    # df['dummy'] = 1
    # df.head(5)
    
    return df

merch_df = pd.read_parquet('../input/merch_df.parquet.gzip')

historical_transactions = add_new_merch_feat(historical_transactions, merch_df)
new_transactions = add_new_merch_feat(new_transactions, merch_df)

del merch_df
gc.collect()

In [ ]:
%%time
# def read_data(input_file):
#     df = pd.read_csv(input_file)
#     df['first_active_month'] = pd.to_datetime(df['first_active_month'])
#     df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
#     return df

# train = read_data('../input/elo-merchant-category-recommendation/train.csv')
# test = read_data('../input/elo-merchant-category-recommendation/test.csv')

def read_data_v2(input_file):
    df = pd.read_parquet(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

train = read_data_v2('../input/train_df.parquet.gzip')
test = read_data_v2('../input/test_df.parquet.gzip')

target = train['target']
del train['target']
gc.collect()

In [ ]:
historical_transactions.head()

## **Feature Engineering**

In [ ]:
plot_opt = 0

In [ ]:
historical_transactions["category_1"] += 1
historical_transactions["category_1"].unique()

In [ ]:
new_transactions["category_1"] += 1
new_transactions["category_1"].unique()

In [ ]:
historical_transactions["category_2"].unique()

In [ ]:
historical_transactions["category_2"] = historical_transactions["category_2"].fillna(0)
historical_transactions["category_2"].unique()

In [ ]:
new_transactions["category_2"].unique()

In [ ]:
new_transactions["category_2"] = new_transactions["category_2"].fillna(0)
new_transactions["category_2"].unique()

In [ ]:
historical_transactions["category_3"].unique()

In [ ]:
historical_transactions["category_3"] = historical_transactions["category_3"].replace({'A': 1, 'B': 2, 'C': 3, None: 0})
historical_transactions["category_3"].unique()

In [ ]:
new_transactions["category_3"].unique()

In [ ]:
new_transactions["category_3"] = new_transactions["category_3"].replace({'A': 1, 'B': 2, 'C': 3, None: 0})
new_transactions["category_3"].unique()

In [ ]:
# historical_transactions["category_1_2_cross"] = historical_transactions["category_1"]*2 + historical_transactions["category_2"]
# np.sort(historical_transactions["category_1_2_cross"].unique())

In [ ]:
# %%time
# if plot_opt:
#     plt.scatter(new_transactions["category_1"], new_transactions["category_2"])
#     plt.xlabel("category_1")
#     plt.ylabel("category_2")
#     plt.show()

In [ ]:
# new_transactions["category_1_2_cross"] = new_transactions["category_1"]*2 + new_transactions["category_2"]
# np.sort(new_transactions["category_1_2_cross"].unique())

In [ ]:
# %%time
# if plot_opt:
#     plt.scatter(historical_transactions["category_1"]+2, historical_transactions["category_3"])
#     plt.xlabel("category_1")
#     plt.ylabel("category_3")
#     plt.show()

In [ ]:
# historical_transactions["category_1_3_cross"] = (historical_transactions["category_1"]+2) * historical_transactions["category_3"]
# np.sort(historical_transactions["category_1_3_cross"].unique())

In [ ]:
# %%time
# if plot_opt:
#     plt.scatter(new_transactions["category_1"], new_transactions["category_3"])
#     plt.xlabel("category_1")
#     plt.ylabel("category_3")
#     plt.show()

In [ ]:
# new_transactions["category_1_3_cross"] = (new_transactions["category_1"]+2) * new_transactions["category_3"]
# np.sort(new_transactions["category_1_3_cross"].unique())

In [ ]:
# %%time
# if plot_opt:
#     plt.scatter(historical_transactions["category_2"]+6, historical_transactions["category_3"])
#     plt.xlabel("category_2")
#     plt.ylabel("category_3")
#     plt.show()

In [ ]:
# historical_transactions["category_2_3_cross"] = (historical_transactions["category_2"]+6) * historical_transactions["category_3"]
# np.sort(historical_transactions["category_2_3_cross"].unique())

In [ ]:
# %%time
# if plot_opt:
#     plt.scatter(new_transactions["category_2"]+6, new_transactions["category_3"])
#     plt.xlabel("category_2")
#     plt.ylabel("category_3")
#     plt.show()

In [ ]:
# new_transactions["category_2_3_cross"] = (new_transactions["category_2"]+6) * new_transactions["category_3"]
# np.sort(new_transactions["category_2_3_cross"].unique())

In [ ]:
%%time

def category_make_cross_feat(df):
    df["category_1_3_cross"] = (df["category_1"]+2) * df["category_3"]
    df["category_2_3_cross"] = (df["category_2"]+6) * df["category_3"]
    return df

historical_transactions = category_make_cross_feat(historical_transactions)
new_transactions = category_make_cross_feat(new_transactions)

In [ ]:
%%time
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 
                                                                           'category_3',
                                                                           'category_1_3_cross',
                                                                           'category_2_3_cross'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 
                                                             'category_3',
                                                             'category_1_3_cross',
                                                             'category_2_3_cross'])

historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

agg_fun = {'authorized_flag': ['sum', 'mean', 'min', 'std', 'count']} # max is all 1's, useless
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]
gc.collect()

In [ ]:
auth_mean.head()

In [ ]:
%%time
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month
gc.collect()

In [ ]:
new_transactions.head()

In [ ]:
%%time
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_1': ['mean'],
        'category_3_2': ['mean'],
        'category_3_3': ['mean'],
        'category_1_3_cross_3': ['mean'],
        'category_1_3_cross_4': ['mean'],
        'category_1_3_cross_6': ['mean'],
        'category_1_3_cross_8': ['mean'],
        'category_1_3_cross_9': ['mean'],
        'category_1_3_cross_12': ['mean'],
        'category_2_3_cross_7.0': ['mean'],
        'category_2_3_cross_8.0': ['mean'],
        'category_2_3_cross_9.0': ['mean'],
        'category_2_3_cross_10.0': ['mean'],
        'category_2_3_cross_11.0': ['mean'],
        'category_2_3_cross_12.0': ['mean'],
        'category_2_3_cross_14.0': ['mean'],
        'category_2_3_cross_16.0': ['mean'],
        'category_2_3_cross_20.0': ['mean'],
        'category_2_3_cross_21.0': ['mean'],
        'category_2_3_cross_22.0': ['mean'],
        'category_2_3_cross_24.0': ['mean'],
        'category_2_3_cross_27.0': ['mean'],
        'category_2_3_cross_30.0': ['mean'],
        'category_2_3_cross_33.0': ['mean'],
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std', 'count'], #one count is enough, others are just the same
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['min', 'max']
        }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

gc.collect()

In [ ]:
%%time
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]
gc.collect()

In [ ]:
# # history[["hist_purchase_amount_count", "hist_installments_count", "hist_purchase_month_count"]].head(500)
# history.head(500)

In [ ]:
%%time
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
authorized[:5]
gc.collect()

In [ ]:
%%time
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]
gc.collect()

In [ ]:
%%time
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
#             'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
#             'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'purchase_amount': ['count', 'sum'],
            'installments': ['count', 'sum'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
#___________________________________________________________
final_group =  aggregate_per_month(historical_transactions) 
final_group[:10]
gc.collect()

In [ ]:
%%time
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test = pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
gc.collect()

In [ ]:
def generate_feat(df):
    # Feature crosses
    df["feature_1_2_cross"] = df["feature_1"] + (df["feature_2"]-1)*5
    df["feature_1_3_cross"] = df["feature_1"] + df["feature_3"]*3
    df["feature_2_3_cross"] = df["feature_2"] + df["feature_3"]*3
    df = pd.get_dummies(df, columns=["feature_1_2_cross", "feature_1_3_cross", "feature_2_3_cross"])
    
    return df

train = generate_feat(train)
test = generate_feat(test)

In [ ]:
def power_2_3_feat(df, feat_list):
    for feat in feat_list:
        df[feat+"_power2"] = df[feat]**2
#         df[feat+"_power3"] = df[feat]**3
    return df

feat_list = ["elapsed_time", "hist_purchase_date_ptp"]
train = power_2_3_feat(train, feat_list)
test = power_2_3_feat(test, feat_list)

In [ ]:
def log_feat(df, feat_list):
    for feat in feat_list:
        df[feat+"_log"] = np.log(df[feat])
#         df[feat+"_power3"] = df[feat]**3
    return df

feat_list = ["elapsed_time"]
train = log_feat(train, feat_list)
test = log_feat(test, feat_list)

In [ ]:
corrmat = train.corr()

In [ ]:
f, ax = plt.subplots(figsize=(100, 100))
sns.heatmap(corrmat, vmax=1.0, square=True)

In [ ]:
f.savefig("../img/corr.png")

In [ ]:
remove_feat_list = ['authorized_flag_min', 
                    'hist_purchase_amount_sum',
                    'hist_purchase_amount_max',
                    'hist_purchase_amount_min',
                    'hist_purchase_amount_std',
                    'hist_installments_sum',
                    'hist_installments_max',
                    'hist_installments_min',
                    'hist_installments_std',
                   ]
train = train.drop(remove_feat_list, axis=1)
test = test.drop(remove_feat_list, axis=1)

In [ ]:
corrmat = train.corr()

In [ ]:
f, ax = plt.subplots(figsize=(100, 100))
sns.heatmap(corrmat, vmax=1.0, square=True)

In [ ]:
f.savefig("../img/corr_after_feat_removal.png")

In [ ]:
# with open('../input/feat_list.pkl', 'rb') as f:
#     feat_list = pickle.load(f)

# train = train[feat_list]
# test = test[feat_list]

#### save train, test

In [ ]:
with open('../input/train_test_target.pkl', 'wb') as f:
    pickle.dump([train, target, test], f)

In [ ]:
with open('../input/train_test_target.pkl', 'rb') as f:
    [train, target, test] = pickle.load(f)

In [ ]:
# https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
# saleprice correlation matrix

# k = train.shape[1] #number of variables for heatmap
# cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
# cm = np.corrcoef(df_train[cols].values.T)
# sns.set(font_scale=1.25)
# hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
# plt.show()

In [ ]:
# #scatterplot
# sns.set()
# # cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
# # sns.pairplot(df_train[cols], size = 2.5)
# sns.pairplot(train, size = 2.5)
# plt.show();

In [ ]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
categorical_feats = [c for c in features if 'feature_' in c]

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

### Outlier removal

In [ ]:
plt.figure(figsize=(12,160))
sns.distplot(target.values, bins=50, kde=False, color='blue')
plt.title('Histogram of Loyalty Score before removal')
plt.xlabel('Loyalty score', fontsize=12)
plt.show()

In [ ]:
from scipy.stats import norm
from scipy import stats
sns.distplot(target, fit=norm)
fig = plt.figure()
res = stats.probplot(target, plot=plt)

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % pd.DataFrame(target).skew())
print("Kurtosis: %f" % pd.DataFrame(target).kurt())

In [ ]:
min_target = min(value for value in target if value > -20)
min_target

In [ ]:
idx_between_20_30 = [value for value in target if value >= -30 and value <=-20]
len(idx_between_20_30)

In [ ]:
idx_lessThan_30 = [value for value in target if value < -30]
len(idx_lessThan_30)

In [ ]:
# https://www.kaggle.com/nottold/naive-ensemble-model-ridge-lasso
class OutlierDetection(BaseEstimator):
    def __init__(self, alpha, dims, std, mean, median):
        self.alpha = alpha
        self.dims = dims
        self.std = std
        self.mean = mean
        self.median = median
    def fit(self, X):
        # std, mean, median = X.std(), X.mean(), X.median()
        X["outliers"] = 0
        for col in X.columns:
#             print(col)
            if not col == "outliers":
                # outlier_idx = (abs(X[col]) > (self.alpha * std[col] + mean[col]))
                outlier_idx = (np.abs(X[col]) > (self.alpha * self.std[col] + self.mean[col]))
                X.set_value(outlier_idx, "outliers", X[outlier_idx]["outliers"] + 1)
        outliers = X[X["outliers"] > self.dims]
        X.drop("outliers", axis=1, inplace=True)
        outlier_idx = outliers.index.tolist()
        # return outliers.index
        return set(list(range(X.shape[0]))) - set(outlier_idx), outlier_idx

In [ ]:
target_df = pd.DataFrame(target)

outlier_removal = OutlierDetection(alpha=3, 
                                   dims=0, 
                                   std=target_df.std().astype('float'), 
                                   mean=target_df.mean().astype('float'), 
                                   median=target_df.median().astype('float'))
normal_idx, outlier_idx = outlier_removal.fit(target_df)
# samples = target_df.shape[0] - len(outlier)
# xtrain = xtrain.drop(outlier_index).reset_index(drop=True)
# y = y.drop(outlier_index).reset_index(drop=True)

In [ ]:
train["outliers"] = 0
train.at[outlier_idx, "outliers"] = 1

In [ ]:
train["outliers"].unique()

In [ ]:
with open('../input/train_test_target_with_target.pkl', 'wb') as f:
    pickle.dump([train, target, test, normal_idx, outlier_idx], f)

In [2]:
with open('../input/train_test_target_with_target.pkl', 'rb') as f:
    [train, target, test, normal_idx, outlier_idx] = pickle.load(f)

## Step 1: Training Model Without Outliers for 5 fold (n_repeats = 1)

In [3]:
df_train = train[train['outliers'] == 0]
df_target = target[normal_idx]
features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers']]
categorical_feats = [c for c in features if 'feature_' in c]

In [4]:
df_train.shape, df_target.shape

((199644, 208), (199644,))

In [5]:
def lgbm_regression_train(train, target, test, param, features, categorical_feats):
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    start = time.time()
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

        num_round = 10000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))
    return predictions, oof

In [6]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005, #default: 0.005 (3.66940)   /   0.005(3.67032), 0.01 (3.67152), 0.05 ()
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "nthread": -1,
         "verbosity": -1}

normal_predictions, oof = lgbm_regression_train(df_train, 
                                                df_target, 
                                                test, 
                                                param, 
                                                features, 
                                                categorical_feats)

fold n°0


/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.62527	valid_1's rmse: 1.64356
[200]	training's rmse: 1.59162	valid_1's rmse: 1.61246
[300]	training's rmse: 1.57277	valid_1's rmse: 1.59586
[400]	training's rmse: 1.56033	valid_1's rmse: 1.58544
[500]	training's rmse: 1.55141	valid_1's rmse: 1.57839
[600]	training's rmse: 1.54427	valid_1's rmse: 1.57304
[700]	training's rmse: 1.53858	valid_1's rmse: 1.56906
[800]	training's rmse: 1.53387	valid_1's rmse: 1.56623
[900]	training's rmse: 1.52979	valid_1's rmse: 1.56395
[1000]	training's rmse: 1.52614	valid_1's rmse: 1.56218
[1100]	training's rmse: 1.52288	valid_1's rmse: 1.56074
[1200]	training's rmse: 1.5199	valid_1's rmse: 1.55964
[1300]	training's rmse: 1.51711	valid_1's rmse: 1.55868
[1400]	training's rmse: 1.51455	valid_1's rmse: 1.55791
[1500]	training's rmse: 1.51211	valid_1's rmse: 1.55729
[1600]	training's rmse: 1.50982	valid_1's rmse: 1.5568
[1700]	training's rmse: 1.50761	valid_1's rmse: 1.55

[3600]	training's rmse: 1.47566	valid_1's rmse: 1.54347
[3700]	training's rmse: 1.47408	valid_1's rmse: 1.54335
[3800]	training's rmse: 1.47257	valid_1's rmse: 1.54334
[3900]	training's rmse: 1.47104	valid_1's rmse: 1.54328
[4000]	training's rmse: 1.46954	valid_1's rmse: 1.54325
[4100]	training's rmse: 1.46797	valid_1's rmse: 1.54317
[4200]	training's rmse: 1.46638	valid_1's rmse: 1.5431
[4300]	training's rmse: 1.46487	valid_1's rmse: 1.54307
[4400]	training's rmse: 1.46333	valid_1's rmse: 1.54303
[4500]	training's rmse: 1.46184	valid_1's rmse: 1.54297
[4600]	training's rmse: 1.46033	valid_1's rmse: 1.54295
[4700]	training's rmse: 1.45884	valid_1's rmse: 1.5429
[4800]	training's rmse: 1.45735	valid_1's rmse: 1.54288
[4900]	training's rmse: 1.45584	valid_1's rmse: 1.54282
[5000]	training's rmse: 1.45436	valid_1's rmse: 1.54278
[5100]	training's rmse: 1.4529	valid_1's rmse: 1.54274
[5200]	training's rmse: 1.45143	valid_1's rmse: 1.54271
[5300]	training's rmse: 1.44995	valid_1's rmse: 1.5

In [7]:
model_without_outliers = pd.DataFrame({"card_id":test["card_id"].values})
model_without_outliers["target"] = normal_predictions

## Part 2: Training Model For Outliers Classification for 5 fold (n_repeats = 1)

In [8]:
def lgbm_classification_train(df_train, target, df_test, param, features, categorical_feats):
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(df_train))
    predictions = np.zeros(len(df_test))
    feature_importance_df = pd.DataFrame()

    start = time.time()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

        num_round = 10000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
        oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
    print("CV score: {:<8.5f}".format(log_loss(target, oof)))

    return predictions, oof

In [9]:
param = {'num_leaves': 51, #default: 31
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333}

with open('../input/train_test_target_with_target.pkl', 'rb') as f:
    [train, target, test, normal_idx, outlier_idx] = pickle.load(f)

target = train['outliers']
del train['outliers']
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
categorical_feats = [c for c in features if 'feature_' in c]
outlier_label, oof_class = lgbm_classification_train(train, target, test, param, features, categorical_feats)

fold n°0


/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0451509	valid_1's binary_logloss: 0.0480669
[200]	training's binary_logloss: 0.0451168	valid_1's binary_logloss: 0.0480732
Early stopping, best iteration is:
[42]	training's binary_logloss: 0.0451386	valid_1's binary_logloss: 0.0480215
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0457537	valid_1's binary_logloss: 0.046939
[200]	training's binary_logloss: 0.0457265	valid_1's binary_logloss: 0.0468905
Early stopping, best iteration is:
[39]	training's binary_logloss: 0.0456993	valid_1's binary_logloss: 0.0468931
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0453821	valid_1's binary_logloss: 0.0466012
[200]	training's binary_logloss: 0.0453995	valid_1's binary_logloss: 0.0465879
Early stopping, best iteration is:
[42]	training's binary_logloss: 0.0453747	valid_1's binary_

In [10]:
df_outlier_prob = pd.DataFrame({"card_id":test["card_id"].values})
df_outlier_prob["target"] = outlier_label
df_outlier_prob.head()

,card_id,target
0,C_ID_0ab67a22ab,0.107459
1,C_ID_130fd0cbdd,0.000801
2,C_ID_b709037bc5,0.010603
3,C_ID_d27d835a9f,0.000683
4,C_ID_2b5e3df5c2,0.000683


## Part 3: Predict outlier for 5 fold (n_repeats = 1)

In [11]:
with open('../input/train_test_target_with_target.pkl', 'rb') as f:
    [train, target, test, normal_idx, outlier_idx] = pickle.load(f)

In [12]:
df_train = train[train['outliers'] == 1]
df_target = target[outlier_idx]
features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers']]
categorical_feats = [c for c in features if 'feature_' in c]

In [13]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005, #default: 0.005 (3.66940)   /   0.005(3.67032), 0.01 (3.67152), 0.05 ()
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "nthread": -1,
         "verbosity": -1}

outlier_predictions, oof_outlier = lgbm_regression_train(df_train, 
                                                         df_target, 
                                                         test, 
                                                         param, 
                                                         features, 
                                                         categorical_feats)

fold n°0
Training until validation scores don't improve for 200 rounds.


/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[100]	training's rmse: 5.63465	valid_1's rmse: 3.97548
[200]	training's rmse: 5.37692	valid_1's rmse: 4.04092
Early stopping, best iteration is:
[29]	training's rmse: 5.90163	valid_1's rmse: 3.93729
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 5.21715	valid_1's rmse: 6.19824
[200]	training's rmse: 5.0162	valid_1's rmse: 6.15378
[300]	training's rmse: 4.83747	valid_1's rmse: 6.11821
[400]	training's rmse: 4.67576	valid_1's rmse: 6.08059
[500]	training's rmse: 4.52147	valid_1's rmse: 6.04763
[600]	training's rmse: 4.3771	valid_1's rmse: 6.02036
[700]	training's rmse: 4.2439	valid_1's rmse: 5.99999
[800]	training's rmse: 4.11679	valid_1's rmse: 5.97422
[900]	training's rmse: 3.99291	valid_1's rmse: 5.96361
[1000]	training's rmse: 3.88143	valid_1's rmse: 5.95469
[1100]	training's rmse: 3.7716	valid_1's rmse: 5.93845
[1200]	training's rmse: 3.66845	valid_1's rmse: 5.92679
[1300]	training's rmse: 3.57088	valid_1's rmse: 5.91566
[1400]	trainin

## Part 4: Combining Submission for 5 fold (n_repeats = 1)

### Using the old way

In [14]:
outlier_id = pd.DataFrame(\
                          df_outlier_prob.sort_values(by='target',
                                                      ascending = False)
                          .head(25000)['card_id'])

In [15]:
# best_submission = pd.read_csv('../result/Blend2_v2.csv')
best_submission = pd.read_csv('../result/Blend2_v10.csv')

In [16]:
print(best_submission.shape[0])
best_submission.head()

123623


,card_id,target
0,C_ID_0ab67a22ab,-2.409111
1,C_ID_130fd0cbdd,-0.360448
2,C_ID_b709037bc5,-0.833417
3,C_ID_d27d835a9f,-0.246980
4,C_ID_2b5e3df5c2,-1.150486


In [17]:
print(outlier_id.shape[0])
most_likely_liers = best_submission.merge(outlier_id,how='right')
most_likely_liers.head()

25000


,card_id,target
0,C_ID_0ab67a22ab,-2.409111
1,C_ID_6d8dba8475,-0.886411
2,C_ID_4859ac9ed5,-0.627507
3,C_ID_7f1041e8e1,-5.065158
4,C_ID_22e4a47c72,0.362193


In [18]:
%%time
for card_id in most_likely_liers['card_id']:
    model_without_outliers.loc[model_without_outliers['card_id']==card_id,'target']\
    = most_likely_liers.loc[most_likely_liers['card_id']==card_id,'target'].values

CPU times: user 4min 23s, sys: 44.9 ms, total: 4min 23s
Wall time: 4min 23s


In [19]:
# model_without_outliers.to_csv("../result/Blend2_v3.csv", index=False)

### Using the combined model

In [20]:
model_without_outliers_new = pd.read_csv('../result/Blend2_v2.csv')

In [21]:
# model_without_outliers_new[] = 
# model_without_outliers_new[] = outlier_predictions

## Step 4: Training Model Without Outliers for 5 fold (n_repeats = 2)

In [22]:
def lgbm_regression_train_n_repeats_2(train, target, test, param, features, categorical_feats):
    folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4520)
    oof_lgb = np.zeros(len(train))
    predictions_lgb = np.zeros(len(test))
    start = time.time()
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

        num_round = 11000
        clf = lgb.train(lgbparam, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
        oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / (5 * 2)

    print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb, target)**0.5))
    return predictions_lgb, oof_lgb

In [23]:
lgbparam = {'num_leaves': 80,
            'min_data_in_leaf': 100, 
            'boosting_type': 'rf',
             'objective':'regression',
             'max_depth': -1,
             'learning_rate': 0.005,
             "min_child_samples": 20,
             "boosting": "gbdt",
             "feature_fraction": 0.9,
             "bagging_freq": 1,
             "bagging_fraction": 0.9 ,
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": -1,
             "random_state": 4590}

# lgbparam = {'num_leaves': 31,
#             'boosting_type': 'rf',
#              'min_data_in_leaf': 30, 
#              'objective':'regression',
#              'max_depth': -1,
#              'learning_rate': 0.005,
#              "min_child_samples": 20,
#              "boosting": "gbdt",
#              "feature_fraction": 0.9,
#              "bagging_freq": 1,
#              "bagging_fraction": 0.9 ,
#              "bagging_seed": 11,
#              "metric": 'rmse',
#              "lambda_l1": 0.1,
#              "verbosity": -1,
#              "nthread": 4,
#              "random_state": 4590} # 1.54083

with open('../input/train_test_target_with_target.pkl', 'rb') as f:
    [train, target, test, normal_idx, outlier_idx] = pickle.load(f)

df_train = train[train['outliers'] == 0]
df_target = target[normal_idx]
features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers']]
categorical_feats = [c for c in features if 'feature_' in c]
    
normal_predictions_n_repeats_2, oof_n_repeats_2 = lgbm_regression_train_n_repeats_2(df_train, 
                                                                                    df_target, 
                                                                                    test, 
                                                                                    lgbparam, 
                                                                                    features, 
                                                                                    categorical_feats)

fold n°0


/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.6162	valid_1's rmse: 1.63259
[200]	training's rmse: 1.57665	valid_1's rmse: 1.59753
[300]	training's rmse: 1.55418	valid_1's rmse: 1.57959
[400]	training's rmse: 1.53928	valid_1's rmse: 1.56914
[500]	training's rmse: 1.52808	valid_1's rmse: 1.56232
[600]	training's rmse: 1.51906	valid_1's rmse: 1.55764
[700]	training's rmse: 1.51142	valid_1's rmse: 1.55434
[800]	training's rmse: 1.50478	valid_1's rmse: 1.55203
[900]	training's rmse: 1.49877	valid_1's rmse: 1.55038
[1000]	training's rmse: 1.49314	valid_1's rmse: 1.54915
[1100]	training's rmse: 1.48792	valid_1's rmse: 1.54826
[1200]	training's rmse: 1.48299	valid_1's rmse: 1.54769
[1300]	training's rmse: 1.47837	valid_1's rmse: 1.54714
[1400]	training's rmse: 1.47394	valid_1's rmse: 1.5468
[1500]	training's rmse: 1.46965	valid_1's rmse: 1.54647
[1600]	training's rmse: 1.46551	valid_1's rmse: 1.54624
[1700]	training's rmse: 1.46157	valid_1's rmse: 1.54

[1400]	training's rmse: 1.47402	valid_1's rmse: 1.54677
[1500]	training's rmse: 1.46978	valid_1's rmse: 1.54653
[1600]	training's rmse: 1.46567	valid_1's rmse: 1.5463
[1700]	training's rmse: 1.4617	valid_1's rmse: 1.54619
[1800]	training's rmse: 1.45779	valid_1's rmse: 1.54614
[1900]	training's rmse: 1.454	valid_1's rmse: 1.54611
[2000]	training's rmse: 1.45029	valid_1's rmse: 1.54603
[2100]	training's rmse: 1.44673	valid_1's rmse: 1.54594
[2200]	training's rmse: 1.44315	valid_1's rmse: 1.54593
[2300]	training's rmse: 1.43963	valid_1's rmse: 1.54585
[2400]	training's rmse: 1.43614	valid_1's rmse: 1.54588
Early stopping, best iteration is:
[2314]	training's rmse: 1.43915	valid_1's rmse: 1.54583
fold n°6
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.61936	valid_1's rmse: 1.61665
[200]	training's rmse: 1.57937	valid_1's rmse: 1.58285
[300]	training's rmse: 1.55684	valid_1's rmse: 1.56634
[400]	training's rmse: 1.54162	valid_1's rmse: 1.55674
[500]

In [24]:
model_without_outliers_n_repeats_2 = pd.DataFrame({"card_id":test["card_id"].values})
model_without_outliers_n_repeats_2["target"] = normal_predictions_n_repeats_2

## Part 5: Training Model For Outliers Classification for 5 fold (n_repeats = 2)

In [25]:
def lgbm_classification_train_n_repeats_2(df_train, target, df_test, param, features, categorical_feats):
    folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=15)
    oof = np.zeros(len(df_train))
    predictions = np.zeros(len(df_test))
    feature_importance_df = pd.DataFrame()

    start = time.time()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

        num_round = 10000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
        oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / (5 * 2)
        
    print("CV score: {:<8.5f}".format(log_loss(target, oof)))

    return predictions, oof

In [26]:
param = {'num_leaves': 51,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.001,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333}

with open('../input/train_test_target_with_target.pkl', 'rb') as f:
    [train, target, test, normal_idx, outlier_idx] = pickle.load(f)

target = train['outliers']
del train['outliers']
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
categorical_feats = [c for c in features if 'feature_' in c]
outlier_label_n_repeats_2, oof_class_n_repeats_2 = lgbm_classification_train_n_repeats_2(train, 
                                                                                         target, 
                                                                                         test, 
                                                                                         param, 
                                                                                         features, 
                                                                                         categorical_feats)

fold n°0


/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/frank/miniconda2/envs/python35/lib/python3.5/site-packages/lightgbm/basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0451509	valid_1's binary_logloss: 0.0480669
[200]	training's binary_logloss: 0.0451168	valid_1's binary_logloss: 0.0480732
Early stopping, best iteration is:
[42]	training's binary_logloss: 0.0451386	valid_1's binary_logloss: 0.0480215
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0457537	valid_1's binary_logloss: 0.046939
[200]	training's binary_logloss: 0.0457265	valid_1's binary_logloss: 0.0468905
Early stopping, best iteration is:
[39]	training's binary_logloss: 0.0456993	valid_1's binary_logloss: 0.0468931
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0453821	valid_1's binary_logloss: 0.0466012
[200]	training's binary_logloss: 0.0453995	valid_1's binary_logloss: 0.0465879
Early stopping, best iteration is:
[42]	training's binary_logloss: 0.0453747	valid_1's binary_

In [27]:
df_outlier_prob_n_repeats_2 = pd.DataFrame({"card_id":test["card_id"].values})
df_outlier_prob_n_repeats_2["target"] = outlier_label_n_repeats_2
df_outlier_prob_n_repeats_2.head()

,card_id,target
0,C_ID_0ab67a22ab,0.102901
1,C_ID_130fd0cbdd,0.000779
2,C_ID_b709037bc5,0.010566
3,C_ID_d27d835a9f,0.000679
4,C_ID_2b5e3df5c2,0.000693


## Part 6: Combining Submission for 5 fold (n_repeats = 2)


In [28]:
outlier_id_n_repeats_2 = pd.DataFrame(\
                                      df_outlier_prob_n_repeats_2.sort_values(by='target',
                                                                              ascending = False)
                                      .head(25000)['card_id'])

In [29]:
# best_submission = pd.read_csv('../result/Blend2_v2.csv')
best_submission = pd.read_csv('../result/Blend2_v10.csv')

In [30]:
print(best_submission.shape[0])
best_submission.head()

123623


,card_id,target
0,C_ID_0ab67a22ab,-2.409111
1,C_ID_130fd0cbdd,-0.360448
2,C_ID_b709037bc5,-0.833417
3,C_ID_d27d835a9f,-0.246980
4,C_ID_2b5e3df5c2,-1.150486


In [31]:
print(outlier_id.shape[0])
most_likely_liers_n_repeats_2 = best_submission.merge(outlier_id_n_repeats_2,how='right')
most_likely_liers_n_repeats_2.head()

25000


,card_id,target
0,C_ID_0ab67a22ab,-2.409111
1,C_ID_6d8dba8475,-0.886411
2,C_ID_4859ac9ed5,-0.627507
3,C_ID_7f1041e8e1,-5.065158
4,C_ID_22e4a47c72,0.362193


In [32]:
%%time
for card_id in most_likely_liers_n_repeats_2['card_id']:
    model_without_outliers_n_repeats_2.loc[model_without_outliers_n_repeats_2['card_id']==card_id,'target']\
    = most_likely_liers_n_repeats_2.loc[most_likely_liers_n_repeats_2['card_id']==card_id,'target'].values

CPU times: user 4min 24s, sys: 54.1 ms, total: 4min 24s
Wall time: 4min 24s


In [33]:
# model_without_outliers_n_repeats_2.to_csv("../result/Blend2_v6_n_repeats_2.csv", index=False)

In [34]:
# len(model_without_outliers['target'].values.tolist())

In [35]:
# len(model_without_outliers_n_repeats_2['target'].values.tolist())

In [36]:
# oof.shape

In [37]:
# oof_n_repeats_2.shape

In [38]:
# target.shape, len(normal_idx), len(outlier_idx)

## Part 7: Stacking

In [39]:
# print(type(target))
# print(type(oof))
# # print(type(oof_normal_final))

In [40]:
# len(outlier_idx), len(outlier_idx)

In [41]:
# oof_normal_final

In [42]:
with open('../input/train_test_target_with_target.pkl', 'rb') as f:
    [train, target, test, normal_idx, outlier_idx] = pickle.load(f)
    
oof_normal_final = pd.Series(np.zeros(len(target)))
oof_normal_final[normal_idx] = oof
oof_normal_final[outlier_idx] = outlier_idx

oof_n_repeats_2_final = pd.Series(np.zeros(len(target)))
oof_n_repeats_2_final[normal_idx] = oof_n_repeats_2
oof_n_repeats_2_final[outlier_idx] = outlier_idx

In [43]:
train_stack = np.vstack([oof_normal_final,oof_n_repeats_2_final]).transpose()
test_stack = np.vstack([model_without_outliers['target'].values.tolist(),
                        model_without_outliers_n_repeats_2['target'].values.tolist()]).transpose()

folds = RepeatedKFold(n_splits=5,n_repeats=1,random_state=4520)
oof_stack = np.zeros(train_stack.shape[0])
predictions_stack = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, target)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    print("-" * 10 + "Stacking " + str(fold_) + "-" * 10)
#     cb_model = CatBoostRegressor(iterations=3000, learning_rate=0.1, depth=8, l2_leaf_reg=20, bootstrap_type='Bernoulli',  eval_metric='RMSE', metric_period=50, od_type='Iter', od_wait=45, random_seed=17, allow_writing_files=False)
#     cb_model.fit(trn_data, trn_y, eval_set=(val_data, val_y), cat_features=[], use_best_model=True, verbose=True)
    clf = BayesianRidge()
    clf.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf.predict(val_data)
    predictions_stack += clf.predict(test_stack) / 5


np.sqrt(mean_squared_error(target.values, oof_stack))

fold n°0
----------Stacking 0----------
fold n°1
----------Stacking 1----------
fold n°2
----------Stacking 2----------
fold n°3
----------Stacking 3----------
fold n°4
----------Stacking 4----------


2.464877040274838

In [51]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission['target'] = predictions_stack
sample_submission.to_csv('../result/Bayesian_Ridge_Stacking.csv', index=False)

In [52]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample1 = pd.read_csv("../result/3.695.csv")
sample2 = pd.read_csv("../result/combining_submission (1).csv")
sample3 = pd.read_csv("../result/single_model_v13_final.csv")
sample4 = pd.read_csv("../result/cttsai_simple_lgbm_cv_bagging_cv_3.648358_lb_3.691.csv")
sample_submission['target'] = model_without_outliers['target'] * 0.5 + model_without_outliers_n_repeats_2['target'] * 0.5
sample_submission.to_csv("../result/3_step_model_final_v11.csv", index = False)
# sample_submission['target'] = sample_submission['target'] * 0.2 + sample1['target'] * 0.2 + sample2['target'] * 0.6
# sample_submission['target'] = sample_submission['target'] * 0.6 + sample1['target'] * 0.2 + sample2['target'] * 0.2
# sample_submission['target'] = sample_submission['target'] / 2 + sample3['target'] / 2
sample_submission['target'] = sample_submission['target'] / 3 + sample3['target'] / 3 + sample4['target'] / 3
# sample_submission.to_csv('../result/special_blend_v3.csv', index=False)
sample_submission.to_csv('../result/special_blending_v5.csv', index=False)

`special_blending_v5.csv` got the best submission score so far - 3.686.